In [1]:
# ans 1
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv("retail_store_sales.csv")

# Basic inspection
print(df.shape)
print(df.dtypes)
df.head()


(12575, 11)
Transaction ID       object
Customer ID          object
Category             object
Item                 object
Price Per Unit      float64
Quantity            float64
Total Spent         float64
Payment Method       object
Location             object
Transaction Date     object
Discount Applied     object
dtype: object


,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,NaN
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False


In [2]:
# ans 2

# Count missing values
missing_values = df.isna().sum().sort_values(ascending=False)
print("Missing Values Per Column:")
print(missing_values)

# Percentage of missing values
missing_percent = (df.isna().mean() * 100).round(2)
print("\nMissing Value Percentage:")
print(missing_percent)

# Check duplicate rows
duplicate_rows = df.duplicated().sum()
print("Duplicate Rows:", duplicate_rows)

# Inspect categorical inconsistencies
categorical_cols = df.select_dtypes(include="object").columns

for col in categorical_cols:
    print("Unique values in :", col)
    print(df[col].value_counts().head(10))

Missing Values Per Column:
Discount Applied    4199
Item                1213
Price Per Unit       609
Total Spent          604
Quantity             604
Transaction ID         0
Customer ID            0
Category               0
Payment Method         0
Location               0
Transaction Date       0
dtype: int64

Missing Value Percentage:
Transaction ID       0.00
Customer ID          0.00
Category             0.00
Item                 9.65
Price Per Unit       4.84
Quantity             4.80
Total Spent          4.80
Payment Method       0.00
Location             0.00
Transaction Date     0.00
Discount Applied    33.39
dtype: float64
Duplicate Rows: 0
Unique values in : Transaction ID
Transaction ID
TXN_2407494    1
TXN_6867343    1
TXN_3731986    1
TXN_9303719    1
TXN_9458126    1
TXN_4575373    1
TXN_7482416    1
TXN_3652209    1
TXN_1372952    1
TXN_9728486    1
Name: count, dtype: int64
Unique values in : Customer ID
Customer ID
CUST_05    544
CUST_24    543
CUST_13    534
CUST_0

In [3]:
# ans 3

df_clean = df.copy()
# Fill categorical columns with mode
for col in categorical_cols:
    df_clean[col] = df_clean[col].fillna(df_clean[col].mode()[0])

# Fill numeric columns with median
numeric_cols = df_clean.select_dtypes(include=np.number).columns
for col in numeric_cols:
    df_clean[col] = df_clean[col].fillna(df_clean[col].median())


for col in categorical_cols:
    df_clean[col] = (
        df_clean[col]
        .astype("string")
        .str.strip()
        .str.lower()
    )


for col in df_clean.columns:
    if "date" in col.lower():
        df_clean[col] = pd.to_datetime(df_clean[col], errors="coerce")


if "price" in df_clean.columns:
    df_clean = df_clean[df_clean["price"] >= 0]

if "quantity" in df_clean.columns:
    df_clean = df_clean[df_clean["quantity"] > 0]


df_clean = df_clean.drop_duplicates()
print("Data cleaning completed successfully.")


Data cleaning completed successfully.


C:\Users\Rudraksh\AppData\Local\Temp\ipykernel_12000\2064649569.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clean[col] = df_clean[col].fillna(df_clean[col].mode()[0])


In [4]:
# ans 4

comparison = pd.DataFrame({
    "Before Cleaning": df.isna().sum(),
    "After Cleaning": df_clean.isna().sum()
})

print(comparison)

print("\nRows Before Cleaning:", df.shape[0])
print("Rows After Cleaning:", df_clean.shape[0])


                  Before Cleaning  After Cleaning
Transaction ID                  0               0
Customer ID                     0               0
Category                        0               0
Item                         1213               0
Price Per Unit                609               0
Quantity                      604               0
Total Spent                   604               0
Payment Method                  0               0
Location                        0               0
Transaction Date                0               0
Discount Applied             4199               0

Rows Before Cleaning: 12575
Rows After Cleaning: 12575


In [5]:
# ans 5

df_clean.to_csv("retail_store_sales_cleaned.csv", index=False)
print("Cleaned dataset saved as retail_store_sales_cleaned.csv")


Cleaned dataset saved as retail_store_sales_cleaned.csv


In [6]:
# ans 6a

if "price" in df_clean.columns:
    Q1 = df_clean["price"].quantile(0.25)
    Q3 = df_clean["price"].quantile(0.75)
    IQR = Q3 - Q1

    df_clean = df_clean[
        (df_clean["price"] >= Q1 - 1.5 * IQR) &
        (df_clean["price"] <= Q3 + 1.5 * IQR)
    ]


In [7]:
# ans 6b

if {"price", "quantity", "category"}.issubset(df_clean.columns):
    df_clean["total_sales"] = df_clean["price"] * df_clean["quantity"]
    sales_summary = (
        df_clean.groupby("category")["total_sales"]
        .sum()
        .sort_values(ascending=False)
    )
    sales_summary
